# M2 and S2 tidal ellipse model

## Plots some tidal ellipses from a set of components by splitting into clockwise and anticlockwise rotating components

In [ ]:
import numpy as np
import xarray as xr
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
# from IPython import display
import seaborn as sns

matplotlib.rcParams['animation.embed_limit'] = 2**128

sns.set()
sns.set_style("dark")
# sns.set_palette("dark")

## make TidalCurrentComp and ComponentSet classes to hold tidal constituent data

In [ ]:
class TidalCurrentComp:
    def __init__(self, name, period, ccw_comp, cw_comp):
        self.name = name
        self.period = period
        self.ccw_comp = ccw_comp
        self.cw_comp = cw_comp
        self.rotate = ["CCW","CW"]
        
    def get_name(self):
        return self.name
    def get_period(self):
        return self.period
    def get_comp(self,rotate):
        if rotate=='ccw':
            return self.ccw_comp
        else:
            return self.ccw_comp
    def timeseries(self,t):
        return xr.DataArray([self.ccw_comp[0]*np.exp(1j*self.ccw_comp[1])*np.exp(1j*2.0*np.pi*(t/self.period))
                             ,self.cw_comp[0]*np.exp(1j*self.cw_comp[1])*np.exp(1j*2.0*np.pi*(-t/self.period))]
                            ,coords=[("rotate",self.rotate),("time",t)]
                            ,dims=["rotate","time"])
        
class ComponentSet:
    def __init__(self):
        self.tides = {}
        
    def add_tide(self,tide):
        self.tides[tide.get_name()]=tide
    
    def list_tides(self):
        return self.tides.keys()
    def tide(self,name):
        return self.tides[name]
        
    

### build set of tidal components

In [ ]:
tides = ComponentSet()
tides.add_tide(TidalCurrentComp('M2',12.42,(1.3,np.pi/4.0),(0.7,np.pi*0.0)))
tides.add_tide(TidalCurrentComp('S2',12.00,(0.45*1.6,np.pi*2.0/3.0),(0.45*0.4,np.pi*2.0/6.0)))

### array of times, in hours, to calculate currents at

In [ ]:
# dimensions, times for calculation
t = np.arange(0, 1000.0, 0.5)

### choose which components to plot - order of selection matters

In [ ]:
# build xarray DataSet of tidal timeseries to plot - use all or specify a subset
td = xr.Dataset()
# for comp in tides.list_tides():
for comp in ['M2','S2']:
    tide = tides.tide(comp)
    td = td.assign({comp:tide.timeseries(t)}) 

### plot and animate

This plots a single frame. Not sure how to stop this happening

In [ ]:
fig,ax = plt.subplots(figsize=(4,5),dpi=100)

i = 0
scale = 0.2
origin = 0 + 1j*0
new_origin = 0 + 1j*0

patches_c = []
patches_a = []

# component arrows and circles
ii=0
for varname, da in td.data_vars.items():
    patches_c.append(plt.Circle((new_origin.real,new_origin.imag),np.abs(da.sel(rotate="CCW").data[0]),fill=False,color='C'+str(ii),lw=2.0))
    patches_a.append(plt.Arrow(new_origin.real,new_origin.imag,da.sel(rotate="CCW").data[i].real,da.sel(rotate="CCW").data[i].imag,
                      width=1.0*scale,color='C'+str(ii)))
    new_origin = new_origin + da.sel(rotate="CCW").data[i]
    ii += 1
    patches_c.append(plt.Circle((new_origin.real,new_origin.imag),np.abs(da.sel(rotate="CW").data[0]),fill=False,color='C'+str(ii),lw=2.0))
    patches_a.append(plt.Arrow(new_origin.real,new_origin.imag,da.sel(rotate="CW").data[i].real,da.sel(rotate="CW").data[i].imag,
                      width=1.0*scale,color='C'+str(ii)))
    new_origin = new_origin + da.sel(rotate="CW").data[i]
    ii += 1
    
for patch in patches_c:
    ax.add_patch(patch)
for patch in patches_a:
    ax.add_patch(patch)

# total velocity arrow
arrv =  plt.Arrow(origin.real,origin.imag,new_origin.real,new_origin.imag,
                      width=1.2*scale,color='xkcd:tomato',zorder=10)
av=ax.add_patch(arrv)

# total velocity tracking point
scat = ax.scatter(new_origin.real,new_origin.imag,color='xkcd:tomato',zorder=10)

# total velocity ellipse trace
x = np.array([new_origin.real])
y = np.array([new_origin.imag])
line, = ax.plot(x,y,zorder=0,color='xkcd:tan',alpha=0.5)

ax.set_aspect('equal')
ax.set_xlim(-3,3)
ax.set_ylim(-3,3)

def animate(i):
    global av,x,y,patches_a

# remove old arrows (arrows cannot be updated)
    av.remove()
    for patch in patches_a:
        patch.remove()
    patches_a = []

# loop over all tidal components
    new_origin = 0 + 1j*0
    ii = 0
    for varname, da in td.data_vars.items():
        
        patches_c[ii].center = (new_origin.real,new_origin.imag)
        patches_a.append(plt.Arrow(new_origin.real,new_origin.imag,da.sel(rotate="CCW").data[i].real,da.sel(rotate="CCW").data[i].imag,
                          width=1.0*scale,color='C'+str(ii)))
        new_origin = new_origin + da.sel(rotate="CCW").data[i]
        ii += 1
        patches_c[ii].center = (new_origin.real,new_origin.imag)
        patches_a.append(plt.Arrow(new_origin.real,new_origin.imag,da.sel(rotate="CW").data[i].real,da.sel(rotate="CW").data[i].imag,
                          width=1.0*scale,color='C'+str(ii)))
        new_origin = new_origin + da.sel(rotate="CW").data[i]
        ii += 1

    for patch in patches_a:
        ax.add_patch(patch)

# total velocity arrow
    arrv =  plt.Arrow(origin.real,origin.imag,new_origin.real,new_origin.imag,
                          width=1.2*scale,color='xkcd:tomato',zorder=10)
    av=ax.add_patch(arrv)

# total velocity tracking point
    scat.set_offsets(np.c_[new_origin.real,new_origin.imag])

    # total velocity ellipse trace
    x = np.append(x,new_origin.real)
    y = np.append(y,new_origin.imag)
    line.set_xdata(x)
    line.set_ydata(y)

    return scat,line

## Set up the animation.

In [ ]:
anim = FuncAnimation(
    fig, animate,frames = 1000,interval = 100,blit=True)

## Run the animation. This will take some time to build...be patient...

In [ ]:
HTML(anim.to_jshtml())